In [1]:
import pandas as pd             
import numpy as np

import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   

import matplotlib.pyplot as plt
%matplotlib inline

import warnings 
warnings.filterwarnings("ignore")

import sklearn
from sklearn import linear_model
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score

#from aux_fun import evaluate

import csv

In [2]:
df=pd.read_csv('train.csv')

In [3]:
from datetime import date
import datetime

df['Day'] = df['DATE'].apply(lambda x: 
                      (date(datetime.datetime.strptime(x,"%Y-%m-%d").year,datetime.datetime.strptime(x,"%Y-%m-%d").month,
                      datetime.datetime.strptime(x,"%Y-%m-%d").day)-date(2019,4,10)).days)
df.drop(['DATE'],axis=1,inplace=True)


In [4]:
site_IDs=df['SITE_ID']
Dates=df['Day']
N_Transported_Sites=df[ 'N_TRANSPORTED_SITES']
Cell_Type=df.loc[:, 'CELL_TYPE_Macro': 'CELL_TYPE_micro']
Geographical_Cluster=df.loc[:, 'GEOGRAPHIC_CLUSTER_K_0': 'GEOGRAPHIC_CLUSTER_K_9']
m3_w_prevXd=df.loc[ :, 'mean_temperature_prev7d':  'min_pressure_prev3d']
m3_w_f_nextXd=df.loc[:,'mean_temperature_f_next14d':'min_pressure_f_next7d']
aircon_sum=df.loc[:,'aircon_sum_wo_prev7d':'aircon_sum_target_next14d']
cat_sum_alarms_prevXd=df.loc[:,'equipment_sum_alarms_prev14d':'temperature_sum_alarms_prev3d']
cat_m3_persistance_prevXd=df.loc[:,'equipment_max_persistance_prev7d':'temperature_min_persistance_prev3d']
skew_cat_alarms_prev14d=df.loc[:,'skew_equipment_alarms_prev14d': 'skew_temperature_alarms_prev14d']
kurt_cat_alarms_prev14d=df.loc[:,'kurt_equipment_alarms_prev14d': 'kurt_temperature_alarms_prev14d']
target_data=df.loc[:,'aircon_sum_target_next14d']

In [5]:
target_variable='aircon_sum_target_next14d'
selected_features= ('SITE_ID','CELL_TYPE_micro', 'GEOGRAPHIC_CLUSTER_K_1', 'GEOGRAPHIC_CLUSTER_K_3',
       'GEOGRAPHIC_CLUSTER_K_7', 'GEOGRAPHIC_CLUSTER_K_9',
       'aircon_sum_wo_prev7d', 'aircon_sum_wo_prev14d', 'mean_rain_mm_prev7d',
       'mean_temperature_f_next14d', 'max_temperature_f_next14d',
       'min_temperature_f_next14d', 'mean_temperature_f_next7d',
       'max_temperature_f_next7d', 'min_temperature_f_next7d', 'Day')
#input_variables=df.columns[df.columns!=target_variable]
input_variables=selected_features
input_variables=input_variables

X=df.loc[:,input_variables]
y=df.loc[:,target_variable]
X


,SITE_ID,CELL_TYPE_micro,GEOGRAPHIC_CLUSTER_K_1,GEOGRAPHIC_CLUSTER_K_3,GEOGRAPHIC_CLUSTER_K_7,GEOGRAPHIC_CLUSTER_K_9,aircon_sum_wo_prev7d,aircon_sum_wo_prev14d,mean_rain_mm_prev7d,mean_temperature_f_next14d,max_temperature_f_next14d,min_temperature_f_next14d,mean_temperature_f_next7d,max_temperature_f_next7d,min_temperature_f_next7d,Day
0,146,0,0,0,0,0,0.0,0.0,1.33,8.00,12.0,5.0,6.14,9.0,5.0,0
1,146,0,0,0,0,0,0.0,0.0,1.90,8.50,16.0,5.0,5.86,8.0,5.0,1
2,146,0,0,0,0,0,0.0,0.0,4.70,9.36,20.0,5.0,6.29,11.0,5.0,2
3,146,0,0,0,0,0,0.0,0.0,4.77,10.57,22.0,5.0,6.86,11.0,5.0,3
4,146,0,0,0,0,0,0.0,0.0,5.39,11.50,22.0,5.0,7.86,12.0,5.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621295,1251,0,0,0,0,1,0.0,0.0,3.57,5.71,11.0,1.0,6.71,11.0,2.0,295
621296,1251,0,0,0,0,1,0.0,0.0,3.89,5.79,11.0,1.0,6.57,11.0,2.0,296
621297,1251,0,0,0,0,1,0.0,0.0,4.04,5.64,11.0,1.0,5.43,11.0,1.0,297
621298,1251,0,0,0,0,1,0.0,0.0,4.04,5.43,9.0,1.0,4.29,9.0,1.0,298


#### Test Train Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## feature Selection

In [7]:
rfe = RFE(linear_model.LinearRegression(), 5) #10 is the number of features to consider
fit = rfe.fit(X_train, y_train)


In [8]:
feature_mask = fit.get_support() #mask of the chosen features
feature_mask


array([False,  True, False,  True, False,  True,  True,  True, False,
       False, False, False, False, False, False, False])

In [9]:
feature_selected =X_train.columns[feature_mask]
feature_selected

Index(['CELL_TYPE_micro', 'GEOGRAPHIC_CLUSTER_K_3', 'GEOGRAPHIC_CLUSTER_K_9',
       'aircon_sum_wo_prev7d', 'aircon_sum_wo_prev14d'],
      dtype='object')

### Model Creation

### OLS

In [10]:
X_train.columns

Index(['SITE_ID', 'CELL_TYPE_micro', 'GEOGRAPHIC_CLUSTER_K_1',
       'GEOGRAPHIC_CLUSTER_K_3', 'GEOGRAPHIC_CLUSTER_K_7',
       'GEOGRAPHIC_CLUSTER_K_9', 'aircon_sum_wo_prev7d',
       'aircon_sum_wo_prev14d', 'mean_rain_mm_prev7d',
       'mean_temperature_f_next14d', 'max_temperature_f_next14d',
       'min_temperature_f_next14d', 'mean_temperature_f_next7d',
       'max_temperature_f_next7d', 'min_temperature_f_next7d', 'Day'],
      dtype='object')

In [11]:
regr = linear_model.LinearRegression()

In [12]:
regr.fit(X_train, y_train)

LinearRegression()

In [13]:
y_pred = regr.predict(X_test)


In [14]:
print('Coefficients: \n', regr.coef_)
#The mean squared error


Coefficients: 
 [ 4.58130981e-07 -4.25154837e-03 -1.11026917e-03  7.34646869e-03
 -4.85842886e-04  1.00886238e-02  4.40672608e-02  8.66681827e-02
  3.85881973e-04 -8.27811719e-05  2.04760273e-04  1.42836607e-04
  4.58761429e-05 -1.09706373e-04  1.32055454e-05 -4.54397431e-06]


### Lasso

In [15]:
lasso = linear_model.Lasso(0.01, normalize=True)

In [16]:
lasso.fit(X_train, y_train)

Lasso(alpha=0.01, normalize=True)

In [17]:
y_pred_lasso = lasso.predict(X_test)

In [18]:
y_pred_lasso

array([0.00579833, 0.00579833, 0.00579833, ..., 0.00579833, 0.00579833,
       0.00579833])

### Lasso CV

In [19]:
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1]


In [20]:
lassocv = linear_model.LassoCV(eps=0.001, n_alphas=100, alphas=alphas, fit_intercept=True, normalize=False, max_iter=10000,  cv=10)


In [21]:
lassocv.fit(X_train, y_train)


LassoCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1], cv=10, max_iter=10000)

In [22]:
y_pred_lassocv = lassocv.predict(X_test)


In [23]:
lassocv.alpha_


1e-05

###  for submission

In [24]:
result = pd.DataFrame(X_test['SITE_ID'])
result['Day']=X_test['Day']
result['NoA']=y_pred
result.groupby(['SITE_ID','Day'],as_index=False)['NoA'].sum()

,SITE_ID,Day,NoA
0,0,2,0.004422
1,0,9,0.003799
2,0,12,0.003772
3,0,18,0.004441
4,0,21,0.004321
...,...,...,...
124255,2604,279,-0.000369
124256,2604,287,0.000164
124257,2604,292,-0.000144
124258,2604,295,0.000225


In [25]:
result = pd.DataFrame(X_test['SITE_ID'])
result['Day']=X_test['Day']
result['NoA']=y_pred_lasso
result.groupby(['SITE_ID','Day'],as_index=False)['NoA'].sum()

,SITE_ID,Day,NoA
0,0,2,0.005798
1,0,9,0.005798
2,0,12,0.005798
3,0,18,0.005798
4,0,21,0.005798
...,...,...,...
124255,2604,279,0.005798
124256,2604,287,0.005798
124257,2604,292,0.005798
124258,2604,295,0.005798


In [26]:
result = pd.DataFrame(X_test['SITE_ID'])
result['Day']=X_test['Day']
result['NoA']=y_pred_lassocv
result.groupby(['SITE_ID','Day'],as_index=False)['NoA'].sum()

,SITE_ID,Day,NoA
0,0,2,0.004351
1,0,9,0.003813
2,0,12,0.003787
3,0,18,0.004444
4,0,21,0.004332
...,...,...,...
124255,2604,279,-0.000369
124256,2604,287,0.000145
124257,2604,292,-0.000150
124258,2604,295,0.000203


In [27]:
X_test['NoA']=y_test

In [28]:
#for row in result.iterrows():
    

In [29]:
X_test.head()

,SITE_ID,CELL_TYPE_micro,GEOGRAPHIC_CLUSTER_K_1,GEOGRAPHIC_CLUSTER_K_3,GEOGRAPHIC_CLUSTER_K_7,GEOGRAPHIC_CLUSTER_K_9,aircon_sum_wo_prev7d,aircon_sum_wo_prev14d,mean_rain_mm_prev7d,mean_temperature_f_next14d,max_temperature_f_next14d,min_temperature_f_next14d,mean_temperature_f_next7d,max_temperature_f_next7d,min_temperature_f_next7d,Day,NoA
2302,1551,0,0,0,0,0,0.0,0.0,0.00,13.93,20.0,6.0,11.14,18.0,6.0,202,0
449895,668,0,0,0,0,0,0.0,0.0,0.00,12.71,19.0,5.0,16.14,19.0,11.0,195,0
607485,1851,0,0,1,0,0,0.0,0.0,0.00,3.21,8.0,-3.0,1.14,5.0,-3.0,285,0
213908,1093,0,0,0,1,0,0.0,0.0,8.49,16.93,22.0,12.0,14.43,18.0,12.0,8,0
563429,680,0,0,0,1,0,0.0,0.0,5.67,20.64,26.0,15.0,17.86,20.0,15.0,29,0


In [44]:
# eval_lr=evaluate(X_test,result)
# eval_lasso=evaluate(X_test,result_lasso)
# eval_lassocv=evaluate(X_test,result_lassocv)

NameError: name 'evaluate' is not defined

In [30]:
r2_lr = r2_score(y_test, y_pred)
r2_lasso =r2_score(y_test, y_pred_lasso)
r2_lassocv = r2_score(y_test, y_pred_lassocv)


In [46]:
r2_lr

0.049808088109394966

In [47]:
r2_lasso

-4.390124332065426e-06

In [48]:
r2_lassocv

0.04979402122926502

##### with seleted features

In [31]:
r2_lr

0.048719395846672064

In [32]:
r2_lasso

-4.390124332065426e-06

In [33]:
r2_lassocv

0.04864169081809033